In [2]:
import urllib
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import numpy as np

# Постановка задачи

Собери данные по самым популярным моделям авто, рассмотрим стоимость бу автомобилей. Для начала можно взять несколько моделей: Kia Rio, Kia Seed, Hyndai Solaris, Volkswagen Polo, Skoda Octavia. Данные собирай по крупнейшим городам России. Список городов: Москва, Санкт-Петербург, Новосибирск, Екатеринбург, Казань, Нижний Новгород. По каждой модели собери данные о годе выпуска, цене, пробеге, коробке, приводе, расположении руля, цвете и всём остальном. Сохрани всё в базу данных и собирай ежедневно новую информацию. Создай модель, предсказывающую стоимость по данным характеристикам. 

# Техническое задание

1. Напиши парсер, принимающий на вход город, марку и модель автомобиля и выдающий всю нужную информацию (цену, год выпуска итд) записи по всем автомобилям,которые сейчас продаются
2. Информацию о кадом автомобиле сохраняй в базу данных на SQLITE для удобного хранения.
3. Найди способ ежедневно искать новые записи.
4. Из все данных составь csv файл для обучение модели.
5. Предобработай данные и отбери нужные признаки
6. Построй предсказательную модель.

In [2]:
class CityException(Exception):
    
    def __init__(self, text):
        self.text = text
        

class BrandException(Exception):
    
    def __init__(self, text):
        self.text = text
        

class ModelException(Exception):
    
    def __init__(self, text):
        self.text = text

In [3]:
class AutoruSearcher():
    """Класс для убоной выгрузки html страниц
    с сайта auto.ru
    """
    url = 'https://auto.ru/'
    
    def __init__(self, city=None, brand=None, model=None):
        self.city = city
        self.brand = brand
        self.model = model
        self.last_request = None
      
    def get_city_info(self):
        
        """ Просто для теста и возможно понадобится в будущем """
        try:
            if self.city is None:
                raise CityException('Забыли ввести название города')
        except CityException as ct:
            print(ct)
        except:
            print('Неизвестная ошибка')
        else:
            self.last_request = AutoruSearcher.url+self.city+'/'
            return urllib.request.urlopen(self.last_request).read()

        
    def get_brand_info(self):
        """ Просто для теста и возможно понадобится в будущем """
        try:
            if self.city is None:
                raise CityException('Забыли ввести название города')
            if self.brand is None:
                raise BrandException('Забыли ввести название бренда')
            
        except CityException as ct:
            print(ct)
        except BrandException as br:
            print(br)
        except:
            print('Неизвестная ошибка')
        else:
            self.last_request = AutoruSearcher.url+self.city+'/cars/'+self.brand+'/all/'
            return urllib.request.urlopen(self.last_request).read()
    
    
    def get_model_info(self, model_params=False):
        
        if not model_params:
            model_params = '/all/' 
        try:
            if self.city is None:
                raise CityException('Забыли ввести название города')
            if self.brand is None:
                raise BrandException('Забыли ввести название бренда')
            if self.model is None:
                raise ModelException('Забыли ввести название модели')
            
        except CityException as ct:
            print(ct)
        except BrandException as br:
            print(br)
        except ModelException as ml:
            print(ml)
        except:
            print('Неизвестная ошибка')
        else:
            self.last_request = AutoruSearcher.url+self.city+'/cars/'+self.brand+'/'+self.model+model_params
            return urllib.request.urlopen(self.last_request).read()
        
    def get_last_request(self):
        return self.last_request
        


## Считываем html страницы и создаём объект для парсинга

In [286]:
searcher = AutoruSearcher(city='kazan', brand='hyundai', model='solaris')

html = searcher.get_model_info()

soup = BeautifulSoup(html, 'html.parser')

In [287]:
searcher.get_last_request()

'https://auto.ru/kazan/cars/hyundai/solaris/all/'

## Выгружаем список всех страниц

In [160]:
pages_list = []

span_for_pages = soup.find('span', class_='ControlGroup ControlGroup_responsive_no ControlGroup_size_s ListingPagination-module__pages')
pages_tags = span_for_pages.findAll('a', class_='Button Button_color_whiteHoverBlue Button_size_s Button_type_link Button_width_default ListingPagination-module__page')

for i in range(len(pages_tags)):
    pages_list.append(pages_tags[i].get('href'))

In [161]:
pages_list

['https://auto.ru/kazan/cars/hyundai/solaris/all/?page=2',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=3',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=4',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=5',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=6',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=7',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=8',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=9',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=10',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=19']

## Находим ссылки на все автомобили по нашей марке и модели на нашей странице

In [231]:
links = soup.findAll('a', class_='Link ListingItemTitle-module__link')

href_list = []

for i in range(len(links)):
    href_list.append(links[i].get('href'))

## Выгружаем все ссылки по страницам по одной модели

In [4]:
# Проходим по первой странице и выгружаем сслыки на все остальные
def get_pages_list(searcher):
    html = searcher.get_model_info()
    soup = BeautifulSoup(html, 'html.parser')
    pages_list = []
    span_for_pages = soup.find('span', class_='ControlGroup ControlGroup_responsive_no ControlGroup_size_s ListingPagination-module__pages')
    pages_tags = span_for_pages.findAll('a', class_='Button Button_color_whiteHoverBlue Button_size_s Button_type_link Button_width_default ListingPagination-module__page')

    for i in range(len(pages_tags)):
        pages_list.append(pages_tags[i].get('href'))
    
    pages_list.insert(0, searcher.get_last_request())
    return pages_list

In [24]:
searcher = AutoruSearcher(city='kazan', brand='hyundai', model='solaris')

pages_list = get_pages_list(searcher)
pages_list

['https://auto.ru/kazan/cars/hyundai/solaris/all/',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=2',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=3',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=4',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=5',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=6',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=7',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=8',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=9',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=10',
 'https://auto.ru/kazan/cars/hyundai/solaris/all/?page=19']

In [5]:
# Проходим по всем страницам и выгружаем ссылки на все автомобили 

def get_model_href_list(pages_list):
    href_list = []
    for url in pages_list:
        html = urllib.request.urlopen(url)
        soup = BeautifulSoup(html, 'html.parser')
    
        links = soup.findAll('a', class_='Link ListingItemTitle-module__link')

        for i in range(len(links)):
            href_list.append(links[i].get('href'))
            
    return np.unique(href_list)

In [49]:
href_list = get_model_href_list(pages_list)
href_list[:5]

array(['https://auto.ru/cars/new/group/hyundai/solaris/21796131/21800621/1100061562-77ad3924/',
       'https://auto.ru/cars/new/group/hyundai/solaris/21796131/21800621/1100061564-5da4d3ed/',
       'https://auto.ru/cars/new/group/hyundai/solaris/21796131/21800621/1100214380-980bc414/',
       'https://auto.ru/cars/new/group/hyundai/solaris/21796131/21800621/1100214382-72f01edf/',
       'https://auto.ru/cars/new/group/hyundai/solaris/21796132/21800621/1100124502-cf7f88ff/'],
      dtype='<U85')

## Оформляем всё в датафрейм

In [9]:
unique_href_list = np.unique(href_list)

model_df = pd.DataFrame(columns=['city', 'brand', 'model', 'new/used', 'href'])
model_df['href'] = href_list
model_df['city'] = 'kazan'
model_df['brand'] = 'hyundai'
model_df['model'] = 'solaris'
model_df.head()

,city,brand,model,new/used,href
0,kazan,hyundai,solaris,NaN,https://auto.ru/cars/used/sale/hyundai/solaris...
1,kazan,hyundai,solaris,NaN,https://auto.ru/cars/used/sale/hyundai/solaris...
2,kazan,hyundai,solaris,NaN,https://auto.ru/cars/used/sale/hyundai/solaris...
3,kazan,hyundai,solaris,NaN,https://auto.ru/cars/new/group/hyundai/solaris...
4,kazan,hyundai,solaris,NaN,https://auto.ru/cars/used/sale/hyundai/solaris...


In [6]:
def is_used(href):
    href_string = ' '.join(href.split('/'))
    res = re.search(r'used', href_string)
    if res is None:
        return False
    else:
        return True

In [11]:
model_df['new/used'] = model_df.apply(lambda row: used_or_new(row['href']), axis=1)
model_df.head()

,city,brand,model,new/used,href
0,kazan,hyundai,solaris,0,https://auto.ru/cars/used/sale/hyundai/solaris...
1,kazan,hyundai,solaris,0,https://auto.ru/cars/used/sale/hyundai/solaris...
2,kazan,hyundai,solaris,0,https://auto.ru/cars/used/sale/hyundai/solaris...
3,kazan,hyundai,solaris,1,https://auto.ru/cars/new/group/hyundai/solaris...
4,kazan,hyundai,solaris,0,https://auto.ru/cars/used/sale/hyundai/solaris...


## Считываем нужную информацию по одной модели

In [37]:
href_list[0]

'https://auto.ru/cars/used/sale/hyundai/solaris/1098569884-a662fd6b/'

In [16]:
def get_car_info_row(href, searcher):
    html = urllib.request.urlopen(href)
    soup = BeautifulSoup(html, 'html.parser')
    car_info_tags = soup.find('ul', class_='CardInfo')

    car_info = car_info_tags.findAll('li')
    car_info_list = []

    for info in car_info:
        i = 0
        for el in info:
            if i == 1:
                car_info_list.append(el.get_text())
            i += 1
    car_info_list = [el.replace(u'\xa0', u' ') for el in car_info_list]
    
    price_tags = soup.find('span', class_='OfferPriceCaption__price')
    
    if price_tags is None:
        return None
    else:
        price = price_tags.text.replace(u'\xa0', u' ')
    
    car_info_list.insert(0, searcher.city)
    car_info_list.insert(1, searcher.brand)
    car_info_list.insert(2, searcher.model)
    car_info_list.insert(3, href)
    car_info_list.insert(4, price)
    
    return car_info_list[:17]

In [17]:
car_info  = get_car_info_row(href_list[100], searcher)
car_info

['kazan',
 'kia',
 'rio',
 'https://auto.ru/cars/used/sale/kia/rio/1096626838-d3f9a246/',
 '465 000 ₽',
 '2012',
 '87 000 км',
 'седан',
 'чёрный',
 '1.6 л / 123 л.с. / Бензин',
 '3 567 ₽ / год',
 'механическая',
 'передний',
 'Левый',
 'Не требует ремонта',
 '1 владелец',
 'Оригинал']

## Проходим по одной марке и модели по всем городам

In [8]:
# Список городов в формате auto.ru
CITIES = ('kazan', 'sankt-peterburg', 'moskva', 'novosibirsk', 'ekaterinburg', 'nizhniy_novgorod')

MODELS = (['kia', 'rio'], ['kia', 'ceed'], ['hyundai', 'solaris'], ['volkswagen', 'polo'], ['skoda', 'octavia'])

In [18]:
%%time
all_models_data = pd.DataFrame(columns=['city', 'brand', 'model', 'href', 'price', 'year', 'mileage', 'body_type', 'colour', 
                           'engine_info', 'credit_per_month', 'transmission', 'drive', 'hand_drive', 'condition', 'owners_num',
                          'doc_unique']) #Создаём датафрейм для хранения всех данных

np.random.seed(int(time.time()))

for city in CITIES:
    for model in MODELS:
        sleeping_time = np.random.randint(1, 4) # Делаем задержку, чтобы не перегружать сеть
        time.sleep(sleeping_time)
        
        searcher = AutoruSearcher(city, model[0], model[1])
        print(searcher.city, searcher.brand, searcher.model)
        
        pages_list = get_pages_list(searcher) # Считываем ссылки на все страницы
        href_list = get_model_href_list(pages_list) # С каждой страницы созраняем 
        
        for href in href_list:
            if is_used(href): # Анализируем только бу автомобили
                car_info = get_car_info_row(href, searcher)
                if car_info is not None:
                    all_models_data.loc[all_models_data.shape[0]] = car_info
         
        
all_models_data.to_csv('data/all_models_data.csv', index=False)

kazan kia rio
kazan kia ceed
kazan hyundai solaris
kazan volkswagen polo
kazan skoda octavia
sankt-peterburg kia rio
sankt-peterburg kia ceed
sankt-peterburg hyundai solaris
sankt-peterburg volkswagen polo
sankt-peterburg skoda octavia
moskva kia rio
moskva kia ceed
moskva hyundai solaris
moskva volkswagen polo
moskva skoda octavia
novosibirsk kia rio
novosibirsk kia ceed
novosibirsk hyundai solaris
novosibirsk volkswagen polo
novosibirsk skoda octavia
ekaterinburg kia rio
ekaterinburg kia ceed
ekaterinburg hyundai solaris
ekaterinburg volkswagen polo
ekaterinburg skoda octavia
nizhniy_novgorod kia rio
nizhniy_novgorod kia ceed
nizhniy_novgorod hyundai solaris
nizhniy_novgorod volkswagen polo
nizhniy_novgorod skoda octavia
Wall time: 2h 8min 56s


In [23]:
all_models_data.sample(10)

,city,brand,model,href,price,year,mileage,body_type,colour,engine_info,credit_per_month,transmission,drive,hand_drive,condition,owners_num,doc_unique
7079,ekaterinburg,skoda,octavia,https://auto.ru/cars/used/sale/skoda/octavia/1...,1 644 000 ₽,2017,45 000 км,универсал 5 дв.,синий,1.8 л / 180 л.с. / Бензин,9 000 ₽ / год,роботизированная,полный,Левый,Не требует ремонта,1 владелец,Оригинал
4254,moskva,volkswagen,polo,https://auto.ru/cars/used/sale/volkswagen/polo...,645 000 ₽,2018,62 000 км,седан,белый,1.6 л / 90 л.с. / Бензин,1 080 ₽ / год,механическая,передний,Левый,Не требует ремонта,2 владельца,Оригинал
2100,sankt-peterburg,kia,ceed,https://auto.ru/cars/used/sale/kia/ceed/109636...,320 000 ₽,2011,169 000 км,хэтчбек 5 дв.,бежевый,1.4 л / 109 л.с. / Бензин,3 815 ₽ / год,механическая,передний,Левый,Не требует ремонта,3 или более,Оригинал
7076,ekaterinburg,skoda,octavia,https://auto.ru/cars/used/sale/skoda/octavia/1...,1 020 000 ₽,2017,93 000 км,лифтбек,белый,1.8 л / 180 л.с. / Бензин,5 886 ₽ / год,роботизированная,передний,Левый,Не требует ремонта,1 владелец,Оригинал
7237,nizhniy_novgorod,kia,rio,https://auto.ru/cars/used/sale/kia/rio/1095987...,600 000 ₽,2016,35 000 км,хэтчбек 5 дв.,серый,1.4 л / 107 л.с. / Бензин,3 371 ₽ / год,автоматическая,передний,Левый,Не требует ремонта,2 владельца,Оригинал
3369,moskva,kia,rio,https://auto.ru/cars/used/sale/kia/rio/1100082...,505 000 ₽,2015,132 024 км,седан,белый,1.6 л / 123 л.с. / Бензин,3 075 ₽ / год,автоматическая,передний,Левый,Не требует ремонта,2 владельца,Дубликат
3378,moskva,kia,rio,https://auto.ru/cars/used/sale/kia/rio/1100104...,455 000 ₽,2014,193 820 км,седан,серебристый,1.6 л / 123 л.с. / Бензин,3 075 ₽ / год,механическая,передний,Левый,Не требует ремонта,1 владелец,Оригинал
7375,nizhniy_novgorod,kia,rio,https://auto.ru/cars/used/sale/kia/rio/1099204...,320 000 ₽,2011,136 874 км,седан,оранжевый,1.4 л / 95 л.с. / Бензин,2 138 ₽ / год,механическая,передний,Левый,Не требует ремонта,3 или более,Оригинал
3174,moskva,kia,rio,https://auto.ru/cars/used/sale/kia/rio/1096049...,420 000 ₽,2010,92 000 км,седан,белый,1.4 л / 95 л.с. / Бензин,950 ₽ / год,автоматическая,передний,Левый,Не требует ремонта,3 или более,Оригинал
2562,sankt-peterburg,volkswagen,polo,https://auto.ru/cars/used/sale/volkswagen/polo...,650 000 ₽,2017,44 200 км,седан,белый,1.4 л / 125 л.с. / Бензин,4 375 ₽ / год,роботизированная,передний,Левый,Не требует ремонта,1 владелец,Оригинал


In [25]:
print("Число записей =", all_models_data.shape[0])

Число записей = 8245


In [3]:
all_models_data = pd.read_csv('data/all_models_data.csv')
all_models_data.head(3)

,city,brand,model,href,price,year,mileage,body_type,colour,engine_info,credit_per_month,transmission,drive,hand_drive,condition,owners_num,doc_unique
0,kazan,kia,rio,https://auto.ru/cars/used/sale/kia/rio/1044366...,500 000 ₽,2015,60 500 км,седан,серебристый,1.6 л / 123 л.с. / Бензин,3 444 ₽ / год,механическая,передний,Левый,Не требует ремонта,2 владельца,Оригинал
1,kazan,kia,rio,https://auto.ru/cars/used/sale/kia/rio/1070379...,405 000 ₽,2012,130 000 км,седан,чёрный,1.6 л / 123 л.с. / Бензин,4 305 ₽ / год,механическая,передний,Левый,Не требует ремонта,2 владельца,Оригинал
2,kazan,kia,rio,https://auto.ru/cars/used/sale/kia/rio/1082430...,630 000 ₽,2017,67 000 км,седан,белый,1.6 л / 123 л.с. / Бензин,3 690 ₽ / год,механическая,передний,Левый,Не требует ремонта,1 владелец,Оригинал


In [7]:
all_models_data = all_models_data.rename(columns={'credit_per_month':'annual_car_tax'})
all_models_data.to_csv('data/all_models_data.csv', index=False)